In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import time
import json
import jax.numpy as np
import numpy as onp
import jax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from timecast.learners import AR
from timecast.learners._ar import _ar_predict, _ar_batch_window
from timecast.utils.numpy import ecdf
from timecast.utils.losses import MeanSquareError
import torch
import matplotlib

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

/home/dsuo/miniconda3/envs/toy_flood/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [15]:
from ealstm.gaip import FloodLSTM
from ealstm.gaip import FloodData
from ealstm.gaip.utils import MSE, NSE

from timecast.learners import AR
from timecast.optim import SGD

cfg_path = "/home/dsuo/src/toy_flood/ealstm/runs/run_2503_0429_seed283956/cfg.json"
ea_data = pickle.load(open("../ealstm/runs/run_2503_0429_seed283956/lstm_seed283956.p", "rb"))
flood_data = FloodData(cfg_path)

LR_AR = 1e-5
AR_INPUT_DIM=32
AR_OUTPUT_DIM=1
BATCH_SIZE=1

In [168]:
results = {}
mses = []
nses = []
for X, y, basin in tqdm.tqdm(flood_data.generator(), total=len(flood_data.basins)):
    break
    sgd = SGD(learning_rate=LR_AR, online=False)
    ar = AR(input_dim=AR_INPUT_DIM,
            output_dim=AR_OUTPUT_DIM,
            window_size=flood_data.cfg["seq_length"],
            optimizer=sgd,
            history=X[:flood_data.cfg["seq_length"]],
            fit_intercept=True,
            constrain=False
           )
    # NOTE: difference in indexing convention, so need to pad one row
    X = np.vstack((X[flood_data.cfg["seq_length"]:], np.ones((1, X.shape[1]))))
    Y = np.array(ea_data[basin].qobs).reshape(-1, 1)
    
    Y_lstm = np.array(ea_data[basin].qsim).reshape(-1, 1)
    Y_target = Y - Y_lstm
    
    Y_ar = ar.predict_and_update(X, Y_target, batch_size=1)
    
    Y_hat = Y_lstm + Y_ar
    
    mse = MSE(Y, Y_hat)
    nse = NSE(Y, Y_hat)
    results[basin] = {
        "mse": mse,
        "nse": nse,
        "count": X.shape[0],
        "avg_mse": np.mean(np.array(mses)),
        "avg_nse": np.mean(np.array(nses))
    }
    mses.append(mse)
    nses.append(nse)
    print(basin, mse, nse, np.mean(np.array(mses)), np.mean(np.array(nses)))
    break
    

In [169]:
from flax import nn
from flax import optim

import jax
import jax.numpy as jnp

In [177]:
def batch_window(X, window_size, offset=0):
    num_windows = X.shape[0] - window_size + 1
    return np.swapaxes(np.stack([np.roll(X, shift=-(i + offset), axis=0) for i in range(window_size)]), 0, 1)[:num_windows]

class ARF(nn.Module):
    def apply(self, x, input_features, output_features, window_size, history=None):
        
        self.history = self.state("history", (window_size, input_features), nn.initializers.zeros)
        
        if self.is_initializing():
            self.history.value = np.vstack((self.history.value, history))[history.shape[0]:]
        else:
            self.history.value = np.vstack((self.history.value, x))[x.shape[0]:]
        
        y = nn.DenseGeneral(inputs=self.history.value,
                            features=output_features,
                            axis=(0, 1),
                            batch_dims=(),
                            bias=True,
                            dtype=jnp.float32,
                            kernel_init=nn.initializers.zeros,
                            bias_init=nn.initializers.zeros,
                            precision=None,
                            name="linear"
                           )
        return y

In [178]:
with nn.stateful() as init_state:
    model_def = ARF.partial(input_features=32, output_features=1, window_size=270, history=X[:flood_data.cfg["seq_length"]-1])
    ys, params = model_def.init_by_shape(jax.random.PRNGKey(0), [(1, 32)])
    model = nn.Model(model_def, params)

In [179]:
optim_def = optim.GradientDescent(learning_rate=1e-5)
optimizer = optim_def.create(model)

In [180]:
X_t = X[flood_data.cfg["seq_length"]-1:]
result = []
for x, y in tqdm.tqdm(zip(X_t, Y_target), total=X_t.shape[0]):
    def loss_fn(model):
        y_hat = model(x[None, ...])
        return jnp.square(y - y_hat).mean(), y_hat
    with nn.stateful(init_state) as init_state:
        loss, y_hat, grad = optimizer.compute_gradients(loss_fn)
        optimizer = optimizer.apply_gradient(grad)
        result.append(y_hat)

In [182]:
Y_hat = Y_lstm + np.array(result)
MSE(Y_hat, Y)

DeviceArray(0.59074104, dtype=float32)